In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
import re
import string
from nltk import SnowballStemmer

In [2]:
df= pd.read_csv("twitter.csv")
df

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
...,...,...,...,...,...,...,...
24778,25291,3,0,2,1,1,you's a muthaf***in lie &#8220;@LifeAsKing: @2...
24779,25292,3,0,1,2,2,"you've gone and broke the wrong heart baby, an..."
24780,25294,3,0,3,0,1,young buck wanna eat!!.. dat nigguh like I ain...
24781,25295,6,0,6,0,1,youu got wild bitches tellin you lies


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24783 entries, 0 to 24782
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Unnamed: 0          24783 non-null  int64 
 1   count               24783 non-null  int64 
 2   hate_speech         24783 non-null  int64 
 3   offensive_language  24783 non-null  int64 
 4   neither             24783 non-null  int64 
 5   class               24783 non-null  int64 
 6   tweet               24783 non-null  object
dtypes: int64(6), object(1)
memory usage: 1.3+ MB


In [7]:
df.isnull().sum()

Unnamed: 0            0
count                 0
hate_speech           0
offensive_language    0
neither               0
class                 0
tweet                 0
dtype: int64

In [9]:
## we have zero null values

In [11]:
df["labels"] = df["class"].map({0:"Hate speech", 1: "offensive", 2:"no hate and offensive"})
df

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet,labels
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...,no hate and offensive
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,offensive
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,offensive
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,offensive
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,offensive
...,...,...,...,...,...,...,...,...
24778,25291,3,0,2,1,1,you's a muthaf***in lie &#8220;@LifeAsKing: @2...,offensive
24779,25292,3,0,1,2,2,"you've gone and broke the wrong heart baby, an...",no hate and offensive
24780,25294,3,0,3,0,1,young buck wanna eat!!.. dat nigguh like I ain...,offensive
24781,25295,6,0,6,0,1,youu got wild bitches tellin you lies,offensive


## lets take important columns from the data frame

In [14]:
new_data = df[["tweet", "labels"]]
new_data

,tweet,labels
0,!!! RT @mayasolovely: As a woman you shouldn't...,no hate and offensive
1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,offensive
2,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,offensive
3,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,offensive
4,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,offensive
...,...,...
24778,you's a muthaf***in lie &#8220;@LifeAsKing: @2...,offensive
24779,"you've gone and broke the wrong heart baby, an...",no hate and offensive
24780,young buck wanna eat!!.. dat nigguh like I ain...,offensive
24781,youu got wild bitches tellin you lies,offensive


## Lets remove the noise and clean the data for model

In [17]:
import nltk
from nltk.corpus import stopwords

In [19]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [21]:
stopword=set(stopwords.words('english'))

In [23]:
stopword

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

### Data Cleaning & preprocessing

In [26]:

def clean(text):
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = [word for word in text.split(' ') if word not in stopword] # check if the word is not a stop word, 
    text=" ".join(text) #if not then add it to the text 
    stemmer = SnowballStemmer("english") # using stemmer to get the root word
    text = [stemmer.stem(word) for word in text.split(' ')]
    text=" ".join(text)
    return text

<>:3: SyntaxWarning: invalid escape sequence '\['
<>:4: SyntaxWarning: invalid escape sequence '\S'
<>:8: SyntaxWarning: invalid escape sequence '\w'
<>:3: SyntaxWarning: invalid escape sequence '\['
<>:4: SyntaxWarning: invalid escape sequence '\S'
<>:8: SyntaxWarning: invalid escape sequence '\w'
C:\Users\Admin\AppData\Local\Temp\ipykernel_5272\2033515905.py:3: SyntaxWarning: invalid escape sequence '\['
  text = re.sub('\[.*?\]', '', text)
C:\Users\Admin\AppData\Local\Temp\ipykernel_5272\2033515905.py:4: SyntaxWarning: invalid escape sequence '\S'
  text = re.sub('https?://\S+|www\.\S+', '', text)
C:\Users\Admin\AppData\Local\Temp\ipykernel_5272\2033515905.py:8: SyntaxWarning: invalid escape sequence '\w'
  text = re.sub('\w*\d\w*', '', text)


In [28]:
new_data["tweet"] = new_data["tweet"].apply(clean)
new_data.head()

C:\Users\Admin\AppData\Local\Temp\ipykernel_5272\101086842.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data["tweet"] = new_data["tweet"].apply(clean)


,tweet,labels
0,rt mayasolov woman shouldnt complain clean ho...,no hate and offensive
1,rt boy dat coldtyga dwn bad cuffin dat hoe ...,offensive
2,rt urkindofbrand dawg rt ever fuck bitch sta...,offensive
3,rt cganderson vivabas look like tranni,offensive
4,rt shenikarobert shit hear might true might f...,offensive


In [29]:
new_data["labels"].value_counts()

labels
offensive                19190
no hate and offensive     4163
Hate speech               1430
Name: count, dtype: int64

In [30]:
cv=CountVectorizer()
x=np.array(new_data["tweet"])
y=np.array(new_data["labels"])

In [34]:
new_x = cv.fit_transform(x)

In [36]:
new_x.shape

(24783, 25540)

In [38]:
y

array(['no hate and offensive', 'offensive', 'offensive', ...,
       'offensive', 'offensive', 'no hate and offensive'], dtype=object)

In [40]:
y.shape

(24783,)

## lets split the data to train and test

In [43]:
from sklearn.model_selection import train_test_split

In [45]:
x_train,x_test,y_train,y_test = train_test_split(new_x, y, test_size=0.2, random_state=0)

## Building model

In [48]:
from sklearn.tree import DecisionTreeClassifier
dt= DecisionTreeClassifier()
dt.fit(x_train, y_train)

DecisionTreeClassifier()

In [49]:
dt.score(x_test, y_test)

0.8721000605204761

In [50]:
dt.score(x_train, y_train)

0.9992434177342883

In [54]:
pred = dt.predict(x_test)
pred

array(['Hate speech', 'offensive', 'offensive', ..., 'offensive',
       'offensive', 'offensive'], dtype=object)

In [56]:
from sklearn.metrics import accuracy_score
accuracy_score(pred, y_test)

0.8721000605204761

In [60]:
input_data = input("enter a sentence:- ")
check = cv.transform([input_data]).toarray()
output = dt.predict(check)
print(output)

enter a sentence:-  murder


['no hate and offensive']


In [64]:
input_data = input("enter a sentence:- ")
check = cv.transform([input_data]).toarray()
output = dt.predict(check)
print(output)

enter a sentence:-  kill you


['Hate speech']


In [66]:
input_data = input("enter a sentence:- ")
check = cv.transform([input_data]).toarray()
output = dt.predict(check)
print(output)

enter a sentence:-  type some word


['no hate and offensive']
